In [29]:
import pandas as pd
import numpy as np
from pandas_gbq import gbq

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Для Линии Тренда
from numpy import arange,array,ones
from scipy import stats

In [30]:
from IPython.display import display,HTML
HTML('''<button style="align: center; margin:0 auto; display: block; background-color: #4CAF50;  color: white;  border: 2px solid #4CAF50; /* Green */" onclick="jQuery('.code_cell .input_area').toggle();
    jQuery('.prompt').toggle();"><text style = background-color: #4CAF50;>Нажми меня</text></button>''')

In [82]:
#Входные данные

#  Если эксперимент на RU - Нужно поменять 
#  `movavi---owox-demo.969794
#  На 
#  `movavi---owox-demo.8254485

#даты эксперимента
date1 = '2020-03-01'
date2 = '2020-03-31'

#-------------------------------------------------------------------------------------------------------

query_config = {
    'query': {
        'parameterMode': 'NAMED',
        'queryParameters': [
            {
                'name': 'date1',
                'parameterType': {'type': 'STRING'},
                'parameterValue': {'value': date1}
            },
            {
                'name': 'date2',
                'parameterType': {'type': 'STRING'},
                'parameterValue': {'value': date2}
            }
        ]
    }
} 

In [83]:
query_main = """


With ga_table as 
(
    SELECT distinct 
    fullVisitorId                           as user                        --ID Юзера
    ,PARSE_DATE("%Y%m%d", date)             as date                        --Дата в формате ГГГГ-ММ-ДД, а не в формате ГГГГММДД
    ,visitStartTime + hits.time/1000        as hit_time                    --Временная отметка хита в секундах
    ,LOWER(hits.page.pagePath)              as page                        --Адрес страницы
    ,hits.time                               as hit_time_1
    ,LEAD(hits.time, 1) OVER(PARTITION BY fullVisitorId, visitStartTime ORDER BY (visitStartTime + (hits.time / 1000)))  as hit_time_2   --Следующий хит тайм для этого юзера (нужно для определения среднего времени на странице)

    ,channelGrouping                        as channel
    ,geoNetwork.country                     as country

    ,hits.isExit                            as exit
    ,hits.isEntrance                        as entrance

    FROM
    `movavi---owox-demo.969794.ga_sessions_*`,     --Формат ГГГГММДД
    UNNEST(hits) AS hits
        
    WHERE  
    _TABLE_SUFFIX BETWEEN FORMAT_DATE("%Y%m%d", DATE(@date1)) AND FORMAT_DATE("%Y%m%d", DATE(@date2))          			--Даты эксперимента
    Order by fullVisitorId, hit_time
)


, ga_table_joined as 
(

    SELECT distinct 
    user_1      as ProductPage
    ,date_1     as date
    ,channel
    ,country
    ,exit       as exit
    ,entrance   as entrance
    ,bounce
    ,menu       as menu
    ,PPC
    ,SEO
    ,Direct
    ,time_on_page
    
    ,user_2     as academic_for_schools
    ,user_3     as buy_academic
    ,user_4     as buy_academic_form
    ,user_5     as receive_offer
    ,user_6     as receive_offer_form
    ,user_7     as receive_offer_all
    ,user_8     as receive_offer_all_form
    ,user_9     as academic_remote_offer
    ,user_10    as academic_remote_offer_form
    ,user_11    as resource_hub
    

    FROM
    (
        SELECT *
        FROM

        (
----------------------------------------------------Продуктовая
            SELECT 
            user                        as user_1
            ,CASE
  WHEN LENGTH(CAST((EXTRACT(month FROM date)) AS STRING)) = 1
  THEN CONCAT((CAST((EXTRACT(year FROM date)) AS STRING)),'-0',(CAST((EXTRACT(month FROM date)) AS STRING)))
  ELSE CONCAT((CAST((EXTRACT(year FROM date)) AS STRING)),'-',(CAST((EXTRACT(month FROM date)) AS STRING)))
END as date_1
            ,hit_time
            ,page

            ,channel
            ,country
            ,CASE WHEN exit = TRUE THEN user ELSE NULL END as exit
            ,CASE WHEN entrance = TRUE THEN user ELSE NULL END as entrance
            ,CASE WHEN entrance = TRUE AND exit = TRUE THEN user ELSE NULL END as bounce
            ,CASE WHEN REGEXP_CONTAINS(LOWER(page), 'main_menu') THEN user ELSE NULL END as menu
            
            ,CASE WHEN channel = "Paid Search" THEN user ELSE NULL END as PPC
            ,CASE WHEN channel = "Organic Search" THEN user ELSE NULL END as SEO
            ,CASE WHEN channel = "Direct" THEN user ELSE NULL END as Direct
            
             --Считаем время на странице до след хита. Если это время больше 30-ти минут или меньше 6-ти секунд, то пишем NULL
            ,CASE 
              WHEN (Round((hit_time_2 - hit_time_1)/60000,2) <30) AND (Round((hit_time_2 - hit_time_1)/60000,2) > 0.1)     
              THEN Round((hit_time_2 - hit_time_1)/60000,2)
            ELSE NULL
            END AS time_on_page

            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(LOWER(page), 'edu.movavi.com/index')
   
        ) as Step_1
        
----------------------------------------------------academic-for-schools--------------Переключлка в меню
        LEFT JOIN
        (
            SELECT 
            user                          as user_2
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/academic-for-schools') 
        ) as Step_2
        ON
        Step_1.user_1 = Step_2.user_2 AND Step_1.hit_time <= Step_2.hit_time 
        
----------------------------------------------------/buy-academic--------------первые 3 BuyNow кнопки        
        LEFT JOIN
        (
            SELECT 
            user                          as user_3
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/buy-academic') 
        ) as Step_3
        ON
        Step_3.user_3 = Step_1.user_1 AND Step_1.hit_time <= Step_3.hit_time  
        
----------------------------------------Заполнение формы с------------/buy-academic--------------первые 3 BuyNow кнопки        
        LEFT JOIN
        (
            SELECT 
            user                          as user_4
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form117963270/submitted') 
        ) as Step_4
        ON
        Step_4.user_4 = Step_3.user_3 AND Step_3.hit_time <= Step_4.hit_time  
        
----------------------------------------------------/receive-offer--------------4-5 BuyNow кнопки
        LEFT JOIN
        (
            SELECT 
            user                          as user_5
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer') AND NOT
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer-') 
        ) as Step_5
        ON
        Step_5.user_5 = Step_1.user_1 AND Step_1.hit_time <= Step_5.hit_time 
        
----------------------------------------Заполнение формы с------------/receive-offer--------------4-5 BuyNow кнопки
        LEFT JOIN
        (
            SELECT 
            user                          as user_6
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form110272193/submitted') 
        ) as Step_6
        ON
        Step_6.user_6 = Step_5.user_5 AND Step_5.hit_time <= Step_6.hit_time 
        
----------------------------------------------------/receive-offer-all--------------6 BuyNow кнопка
        LEFT JOIN
        (
            SELECT 
            user                          as user_7
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer-all') 
        ) as Step_7
        ON
        Step_7.user_7 = Step_1.user_1 AND Step_1.hit_time <= Step_7.hit_time 

----------------------------------------Заполнение формы с------------/receive-offer-all--------------6 BuyNow кнопка
        LEFT JOIN
        (
            SELECT 
            user                          as user_8
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form169017050/submitted') 
        ) as Step_8
        ON
        Step_8.user_8 = Step_7.user_7 AND Step_7.hit_time <= Step_8.hit_time 

----------------------------------------------------/academic-remote-offer--------------Поп-ап
        LEFT JOIN
        (
            SELECT 
            user                          as user_9
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/academic-remote-offer') 
        ) as Step_9
        ON
        Step_9.user_9 = Step_1.user_1 AND Step_1.hit_time <= Step_9.hit_time 

----------------------------------------Заполнение формы с------------/academic-remote-offer--------------Поп-ап
        LEFT JOIN
        (
            SELECT 
            user                          as user_10
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form170970217/submitted') 
        ) as Step_10
        ON
        Step_10.user_10 = Step_9.user_9 AND Step_9.hit_time <= Step_10.hit_time

----------------------------------------Resource-hub-----------------------------------
        LEFT JOIN
        (
            SELECT 
            user                          as user_11
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/resource-hub') 
        ) as Step_11
        ON
        Step_11.user_11 = Step_1.user_1 AND Step_1.hit_time <= Step_11.hit_time 
        
    )    

)  




SELECT distinct * 
FROM ga_table_joined
Order by  date desc


"""

In [84]:
df = gbq.read_gbq(query=query_main, dialect='standard',project_id='movavi---owox-demo', configuration=query_config)

Downloading: 100%|███████████████████| 20237/20237 [00:06<00:00, 3340.67rows/s]


In [85]:
pd.set_option('display.max_rows', 100)

In [86]:
df

,ProductPage,date,channel,country,exit,entrance,bounce,menu,PPC,SEO,...,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form,resource_hub
0,4250600723504464289,2020-03,Organic Search,Netherlands,None,None,None,None,None,4250600723504464289,...,None,None,None,None,None,None,None,None,None,None
1,8703461265097050226,2020-03,Organic Search,Turkey,None,None,None,None,None,8703461265097050226,...,None,None,None,None,None,None,None,None,None,None
2,3273928617842584339,2020-03,Display,United States,None,None,None,3273928617842584339,None,None,...,None,None,None,None,None,None,None,3273928617842584339,None,None
3,3573279454595832906,2020-03,Paid Search,India,None,None,None,3573279454595832906,3573279454595832906,None,...,None,None,None,None,None,None,None,None,None,None
4,7552320952147272565,2020-03,Paid Search,Italy,None,None,None,7552320952147272565,7552320952147272565,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20232,8472770746198244498,2020-03,Organic Search,Germany,8472770746198244498,None,None,None,None,8472770746198244498,...,None,None,None,None,None,None,None,None,None,None
20233,8472770746198244498,2020-03,Organic Search,Germany,None,None,None,None,None,8472770746198244498,...,None,None,None,None,None,None,None,None,None,None
20234,8622132223938663914,2020-03,Paid Search,Cyprus,None,None,None,None,8622132223938663914,None,...,8622132223938663914,None,None,None,None,None,None,8622132223938663914,None,None
20235,9082675554136765473,2020-03,Direct,Russia,None,9082675554136765473,None,None,None,None,...,None,None,None,None,None,None,None,9082675554136765473,None,None


In [87]:
df_copy = df.copy()

In [88]:
df = df_copy

## Статистика по юзерам на странице edu.movavi.com по месяцам 
(Вкладка Продуктовая, один из столбиков. Добавляю ручками туда)

In [89]:
df_mean = df.groupby(['date']).mean()
df_mean = df_mean[['time_on_page']]
df_mean.sort_values(by=['date'], ascending=True)

,time_on_page
date,
2020-03,2.217039


In [90]:
df_mean = df.groupby(['channel']).mean()
df_mean = df_mean[['time_on_page']]
df_mean.sort_values(by=['channel'], ascending=True)

,time_on_page
channel,
(Other),1.803806
Direct,2.169512
Display,3.125171
Email,0.765000
Organic Search,2.258327
Paid Search,2.091467
Referral,2.890934
Social,1.715374


## Конверсия с продуктовой в посещения страниц и заполнения форм по месяцам
(Вкладка Заполнение форм)

In [91]:
df_count = df.groupby(['date']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form', 'resource_hub']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['date'], ascending=True)

,ProductPage,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form,resource_hub
date,,,,,,,,,,,
2020-03,9322,238,1486,375,328,103,31,5,2327,727,0


## Конверсия с продуктовой в заполнения форм по каналам
(Вкладка Заполнение форм. Там обычно сразу за несколько месяцев. Если нужно каждый месяц отдельно - это ниже.)

In [92]:
df_count = df.groupby(['channel']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form', 'resource_hub']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['ProductPage'], ascending=True)

,ProductPage,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form,resource_hub
channel,,,,,,,,,,,
Email,1,0,0,0,0,0,0,0,1,0,0
Display,135,3,18,3,7,2,0,0,45,9,0
Social,152,3,20,0,6,1,0,0,34,11,0
(Other),225,7,53,16,9,1,1,0,57,9,0
Referral,695,19,111,29,29,5,6,0,114,32,0
Direct,2326,57,295,71,71,13,10,1,507,155,0
Organic Search,2351,75,465,107,120,40,11,1,543,176,0
Paid Search,3685,94,608,173,128,48,9,3,1099,362,0


## Конверсия с продуктовой в заполнения форм по месяцам и каналам
(Вкладка Заполнение форм. Лишние каналы удаляю)

In [93]:


df_count = df.groupby(['date', 'channel']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form', 'resource_hub']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['date'], ascending=True)

ProductPage  academic_for_schools  buy_academic  \
date    channel                                                           
2020-03 (Other)                 225                     7            53   
        Direct                 2326                    57           295   
        Display                 135                     3            18   
        Email                     1                     0             0   
        Organic Search         2351                    75           465   
        Paid Search            3685                    94           608   
        Referral                695                    19           111   
        Social                  152                     3            20   

                        buy_academic_form  receive_offer  receive_offer_form  \
date    channel                                                                
2020-03 (Other)                        16              9                   1   
        Direct                         71             71                  13   
        Display                         3              7                   2   
        Email                           0              0                   0   
        Organic Search                107            120                  40   
        Paid Search                   173            128                  48   
        Referral                       29             29                   5   
        Social                          0              6                   1   

                        receive_offer_all  receive_offer_all_form  \
date    channel                                                     
2020-03 (Other)                         1                       0   
        Direct                         10                       1   
        Display                         0                       0   
        Email                           0                       0   
        Organic Search                 11                       1   
        Paid Search                     9                       3   
        Referral                        6                       0   
        Social                          0                       0   

                        academic_remote_offer  academic_remote_offer_form  \
date    channel                                                             
2020-03 (Other)                            57                           9   
        Direct                            507                         155   
        Display                            45                           9   
        Email                               1                           0   
        Organic Search                    543                         176   
        Paid Search                      1099                         362   
        Referral                          114                          32   
        Social                             34                          11   

                        resource_hub  
date    channel                       
2020-03 (Other)                    0  
        Direct                     0  
        Display                    0  
        Email                      0  
        Organic Search             0  
        Paid Search                0  
        Referral                   0  
        Social                     0

## Конверсия с продуктовой в заполнения форм по странам 
(Вкладка Заполнение форм, обычно там данные за несколько месяцев разом
Если нужно каждый месяц отдельно - это ниже)

In [94]:
df_count = df.groupby(['country']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form', 'resource_hub']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
pd.set_option('display.max_rows', 1000)
df_count.sort_values(by=['ProductPage'], ascending=False)

,ProductPage,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form,resource_hub
country,,,,,,,,,,,
United States,2587,59,474,160,111,47,6,2,866,401,0
India,758,15,126,32,15,6,1,0,134,23,0
Brazil,400,10,67,16,16,4,2,0,135,38,0
United Kingdom,322,12,76,21,15,5,0,0,86,23,0
Australia,281,8,48,19,10,3,0,0,88,29,0
Turkey,256,12,42,7,11,4,2,0,76,18,0
Russia,244,19,25,1,10,0,5,0,37,1,0
China,242,0,21,3,3,0,0,0,19,2,0
Canada,232,8,49,14,10,2,2,1,59,19,0


Тоже самое, что выше, только для каждого месяца отдельно

In [95]:
df_count = df.groupby(['date', 'country']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form', 'resource_hub']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['date'], ascending=True)

ProductPage  academic_for_schools  buy_academic  \
date    country                                                                 
2020-03 (not set)                      18                     1             3   
        Netherlands                    83                     3            18   
        New Caledonia                   1                     0             1   
        New Zealand                    29                     1             5   
        Niger                           1                     0             0   
        Nigeria                        11                     0             2   
        North Macedonia                 5                     0             0   
        Nepal                           5                     0             0   
        Norway                         45                     0             6   
        Pakistan                       86                     2             7   
        Palestine                       9                     0             1   
        Panama                          6                     0             0   
        Paraguay                        1                     0             0   
        Peru                           14                     0             0   
        Philippines                    74                     0             7   
        Oman                           32                     1             3   
        Namibia                         1                     0             0   
        Myanmar (Burma)                 7                     0             0   
        Morocco                        24                     0             0   
        Lebanon                        30                     0             2   
        Libya                           3                     0             1   
        Lithuania                      19                     0             4   
        Luxembourg                      4                     0             1   
        Macao                           5                     0             2   
        Madagascar                      1                     0             0   
        Malaysia                      162                     4            20   
        Malta                           5                     0             0   
        Martinique                      2                     0             0   
        Mauritius                       3                     0             0   
        Mayotte                         1                     0             0   
        Mexico                        134                     1            19   
        Moldova                         2                     0             0   
        Mongolia                        3                     1             1   
        Montenegro                      5                     0             0   
        Poland                         70                     1            16   
        Latvia                          9                     1             4   
        Portugal                       64                     2            11   
        Qatar                          15                     1             4   
        Thailand                      150                     6            22   
        Trinidad & Tobago               4                     0             0   
        Tunisia                         4                     0             0   
        Turkey                        256                    12            42   
        U.S. Virgin Islands             1                     0             0   
        Uganda                          3                     0             0   
        Tanzania                        4                     0             0   
        Ukraine                        39                     1             1   
        United Kingdom                322                    12            76   
        United States                2587                    

#### Вспомогательные данные. В док не выводим

In [96]:
df_count = df.groupby(['date']).nunique()
df_count = df_count[['ProductPage', 'exit', 'entrance', 'bounce', 'menu']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['date'], ascending=True)

,ProductPage,exit,entrance,bounce,menu
date,,,,,
2020-03,9322,3814,2028,1328,7184


## Статистика по юзерам на странице edu.movavi.com по месяцам
(Вкладка продуктовая. Плюс доп столбик нужен, который выше)

In [97]:
df_count = df.groupby(['date']).count()
df_count = df_count[['ProductPage', 'exit', 'entrance', 'bounce', 'menu', 'PPC', 'SEO', 'Direct', 'time_on_page']]
df_count = df_count.assign(Exit_percent=round((df_count['exit'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Entrance_percent=round((df_count['entrance'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Bounce_rate=round((df_count['bounce'] / df_count['entrance']),4)*100)
df_count = df_count.assign(Menu_Percent=round((df_count['menu'] / df_count['ProductPage']),4)*100)

df_count = df_count.assign(PPC_Percent=round((df_count['PPC'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(SEO_Percent=round((df_count['SEO'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Direct_Percent=round((df_count['Direct'] / df_count['ProductPage']),4)*100)

df_count = df_count[['ProductPage', 'Exit_percent', 'Entrance_percent', 'Bounce_rate', 'Menu_Percent', 'PPC_Percent', 'SEO_Percent', 'Direct_Percent']]
df_count.sort_values(by=['date'], ascending=True)

,ProductPage,Exit_percent,Entrance_percent,Bounce_rate,Menu_Percent,PPC_Percent,SEO_Percent,Direct_Percent
date,,,,,,,,
2020-03,20237,20.12,11.24,60.88,70.2,38.15,26.72,22.26


## Статистика по юзерам на странице edu.movavi.com  по каналам
(Вкладка Продуктовая. Обычно тут сразу за несколько месяцев.)

In [98]:
df_count = df.groupby(['channel']).count()
df_count = df_count[['ProductPage', 'exit', 'entrance', 'bounce', 'menu', 'PPC', 'SEO', 'Direct', 'time_on_page']]
df_count = df_count.assign(Exit_percent=round((df_count['exit'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Entrance_percent=round((df_count['entrance'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Bounce_rate=round((df_count['bounce'] / df_count['entrance']),4)*100)
df_count = df_count.assign(Menu_Percent=round((df_count['menu'] / df_count['ProductPage']),4)*100)

df_count = df_count.assign(PPC_Percent=round((df_count['PPC'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(SEO_Percent=round((df_count['SEO'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Direct_Percent=round((df_count['Direct'] / df_count['ProductPage']),4)*100)

df_count = df_count[['ProductPage', 'Exit_percent', 'Entrance_percent', 'Bounce_rate', 'Menu_Percent']]#, 'PPC_Percent', 'SEO_Percent', 'Direct_Percent']]
df_count.sort_values(by=['ProductPage'], ascending=False)

,ProductPage,Exit_percent,Entrance_percent,Bounce_rate,Menu_Percent
channel,,,,,
Paid Search,7720,16.18,1.98,47.06,87.42
Organic Search,5407,19.22,16.16,48.51,56.32
Direct,4504,26.15,16.74,70.16,67.30
Referral,1648,25.61,24.82,71.88,37.56
(Other),441,14.74,1.36,66.67,87.07
Display,259,11.97,1.93,20.00,85.33
Social,255,34.12,29.02,82.43,60.78
Email,3,0.00,0.00,NaN,100.00
